# Grade prediction with BERT embedding

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the reviews we scrapped in previous courses. We will try to predict wether it was positive or negative. 

Under the hood, the model is actually made up of two model.


1.   DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
2.   The next model, a basic Logistic Regression model from scikit learn will identify if the review was positive or negative



## Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [ ]:
# The transformers library is already installed
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from scipy.stats import loguniform
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

We'll import the dataset as we did in previous notebooks:

In [ ]:
reviews = pd.read_csv('reviews.csv')
reviews.drop(columns=["Unnamed: 0"], inplace=True)

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37294 entries, 0 to 37293
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   page             37294 non-null  int64 
 1   titre            37294 non-null  object
 2   verbatim         36314 non-null  object
 3   date             37294 non-null  object
 4   note             37294 non-null  int64 
 5   reponse          7972 non-null   object
 6   date_experience  37294 non-null  object
 7   fournisseur      37294 non-null  object
 8   source           37294 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.6+ MB


In the first place we should determine what are we considering a positive review. In this case we shall consider as positive every comment that had 4 or 5 as grade and negative otherwise.

In [ ]:
# ASSIGN A LABEL TO EACH REVIEW TO DIFFERENCITATE POSITIVE AND NEGATIVE REVIEWS
reviews['sentiment'] = reviews.note.apply(lambda x: 'pos' if x >= 4 else 'neg')

# REMOVE REVIEWS WITH NULL VALUES
reviews.dropna(inplace=True)

For computational purposes, we'll only use 1000 sentences which have less than 100 words in it.

In [ ]:
# Filter the reviews
reviews['verbatim'] = reviews.verbatim.apply(lambda x: x if len(x.split(' ')) <= 100 else np.nan)
reviews.dropna(inplace=True)
reviews = reviews.sample(1000)
reviews.reset_index(inplace=True, drop=True)
reviews

,page,titre,verbatim,date,note,reponse,date_experience,fournisseur,source,sentiment
0,208,honteux,"Tres bien et facile pour obtenir un contrat, p...",27 mai 2020,1,"Bonjour Marina,\n\nJe suis navré d'apprendre q...",Date de l'expérience: 27 mai 2020,https://fr.trustpilot.com/review/totalenergies.fr,trustpilot,neg
1,61,Bon service,"Ohm , services clients satisfaisant.",19 janv. 2022,5,"Bonjour,\nNous sommes très touchés que votre p...",Date de l'expérience: 19 janvier 2022,https://fr.trustpilot.com/review/ohm-energie.com,trustpilot,pos
2,313,Avis client,J'ai etait prévenu assez tard qu'il fallait un...,le 06/10/2022 par ELODIE D.,2,"Bonjour,",suite à une expérience du 27/09/2022,https://www.avis-verifies.com/avis-clients/tot...,avis_verifies,neg
3,22,Avis client,C'est très bien je suis très à l'aise avec eux,le 26/09/2021 par ZAKARIA H.,5,"Bonjour,",suite à une expérience du 24/04/2021,https://www.avis-verifies.com/avis-clients/fr....,avis_verifies,pos
4,358,Avis client,"Prix bien trop cher, montages financiers douteux.",le 02/10/2022 par ALEXANDRE V.,1,"Commentaire de TotalEnergies\nBonjour,\n\nJ'ai...",suite à une expérience du 23/09/2022,https://www.avis-verifies.com/avis-clients/tot...,avis_verifies,neg
...,...,...,...,...,...,...,...,...,...,...
995,405,Avis client,...,le 29/09/2022 par RAPHAËLLE B.,5,"Bonjour,",suite à une expérience du 16/09/2022,https://www.avis-verifies.com/avis-clients/tot...,avis_verifies,pos
996,347,Avis client,Horrible,le 03/10/2022 par ANISSA L.,1,"Bonjour Madame,",suite à une expérience du 23/09/2022,https://www.avis-verifies.com/avis-clients/tot...,avis_verifies,neg
997,103,Bonne Expérience avec Ohm Energie,Bonne Expérience avec Ohm Energie . Bonne acco...,14 juin 2021,5,"Bonjour,\nNous sommes ravis que nos services a...",Date de l'expérience: 14 juin 2021,https://fr.trustpilot.com/review/ohm-energie.com,trustpilot,pos
998,461,Avis client,"L'offre que j'ai souscrite ""essentielle"" est u...",le 25/09/2022 par GREGORY B.,2,"Bonjour,",suite à une expérience du 10/09/2022,https://www.avis-verifies.com/avis-clients/tot...,avis_verifies,neg


In [ ]:
#Save in two variables the comments and the labels
sentences = reviews['verbatim'].values
labels = reviews['sentiment'].values

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Right now, the variable model holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

# Model 1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
# Apply tokenization
tokenized = [tokenizer(sentence) for sentence in sentences]

### Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).


In [ ]:
# Perform the padding
max_len = 0

for t in tokenized:
    if len(t['input_ids']) > max_len:
        max_len = len(t['input_ids'])

padded = np.array([t['input_ids'] + [0] * (max_len - len(t['input_ids'])) for t in tokenized])

Our dataset is now in the padded variable, we can view its dimensions below:

In [ ]:
padded.shape

(1000, 217)

### Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 217)

## Model 1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model! 

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the features variable, as they'll serve as the features to our logitics regression model.



In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

## Model 2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 1,000 sentences from the reviews training set).

In [ ]:
# Split the data in train and test with the train_test_split function
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=0)

## [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [ ]:
param_grid = {'C': loguniform.rvs(1e-05, 100, size=200)} # using a log uniform distribution to test different orders of magnitude
logreg = LogisticRegression()
logreg_grid = GridSearchCV(logreg, param_grid, cv=3)
logreg_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': array([3.41349058e+00, 8.12125219e-05, 2.79301940e+00, 4.82111370e-02,
       7.33725967e-03, 6.84815584e-03, 3.56461660e-04, 4.56614485e-04,
       1.31019690e-04, 2.25346474e+01, 5.17810369e-05, 2.27373696e-02,
       1.28920093e+00, 1.39458291e-03, 2.73032607e-02, 6.19396894e-04,
       9.37874966e-02, 7.13676378e+01, 2.549970...
       1.80541839e-05, 1.38445205e+00, 2.42294898e-05, 1.92992532e+00,
       3.29686208e+01, 5.28199561e-03, 3.13283014e-05, 1.70559678e-02,
       8.00723392e+01, 4.15799422e-02, 5.49151100e-03, 2.71379372e+01,
       3.28577885e-02, 1.76598278e-01, 2.41057562e-05, 1.84555548e-05,
       2.42000292e-05, 6.84486965e-03, 4.19723119e+00, 2.74593676e-02,
       2.38679011e+00, 7.37128542e-02, 9.39987999e-02, 4.96891052e-02])})

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. LogisticRegression(C=5.2)).

In [ ]:
# Fit the logistic regression model
logreg_model = logreg_grid.best_estimator_
logreg_model

LogisticRegression(C=3.4134905785885077)

## Evaluating Model 2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
# Evaluate the score of the model
y_pred = logreg_model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {round(accuracy * 100, 2)}%")

Accuracy: 83.5%


How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

# Compare your model with a dummy classifier
clf.fit(X_train, y_train)
clf_pred = clf.predict(X_test)
clf_accuracy = np.mean(clf_pred == y_test)
print(f"Accuracy: {round(clf_accuracy * 100, 2)}%")

Accuracy: 67.0%


So our model clearly does better than a dummy classifier